In [89]:
#-*- coding:utf-8 -*-
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transform
import numpy as np
import pandas as pd
import torch.utils.data as data

train = pd.read_csv("data.csv",parse_dates=["Date"])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [90]:
# Hyper-Parameters
sequence_length = 20
input_size = 2
hidden_size = 128
num_layers = 2
output_size = 1
batch_size = 100
num_epochs = 200
learning_rate = 5e-5

In [91]:
##Build RNN(LSTM)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size,output_size)

    def forward(self, x):
        # set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [92]:
input1 = np.array(train["GridPower_ma"][19:])
input2 = np.array(train["WindSpd_ma"][19:])
input = np.vstack((input1,input2)).T
input = torch.tensor(input)
gridpower = np.array(train["GridPower_ma"][19:])
gridpower = torch.tensor(gridpower)

In [93]:
class mydata(data.Dataset):
    def __init__(self, input_dat, y):
        self.input_dat = input_dat
        self.y = y
        self.idx = list()
        for item in input_dat:
            self.idx.append(item)
        pass

    def __getitem__(self, index):
        value = self.idx[index]
        label = self.y[index]
        return value,label
    
    def __len__(self):
        return len(self.idx)

In [94]:
# Data Loader
train_data = mydata(input,gridpower)
train_dat = data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

In [95]:
model = RNN(input_size, hidden_size, num_layers, output_size).to(device)


# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    # forward pass
    outputs = model(input)
    loss = criterion(outputs,gridpower)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(loss.item())

In [96]:
# Train the model
for epoch in range(num_epochs):
    for i, (value,labels) in enumerate(train_dat):
        value = value.reshape(-1,sequence_length,input_size).to(device)
        value = value.to(torch.float32)
        labels = labels.to(device)
        labels = labels.to(torch.float32)

        # forward pass
        outputs = model(value)
        loss = criterion(outputs.squeeze(),labels)

        # optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(loss.item())

129.34706115722656
118.79779052734375
101.02709197998047
118.80011749267578
152.89306640625
98.8016586303711
143.74554443359375
98.1234359741211
116.13731384277344
90.4646224975586
119.52916717529297
99.6573257446289
98.36796569824219
96.89983367919922
78.20830535888672
95.28868103027344
73.05232238769531
63.27443313598633
81.84185791015625
43.455997467041016
43.26817321777344
52.42966842651367
46.25678634643555
32.62325668334961
45.36058044433594
55.456390380859375
54.60381317138672
53.67306137084961
59.88694763183594
46.65523910522461
41.61966323852539
35.85218811035156
33.14362335205078
38.68270492553711
33.48432540893555
40.159454345703125
40.37846755981445
32.26632308959961
38.50882339477539
24.46969985961914
30.948535919189453
20.255390167236328
16.36689567565918
25.948305130004883
24.584245681762695
27.130399703979492
24.562837600708008
22.951000213623047
27.587879180908203
28.739683151245117
18.506635665893555
22.646390914916992
23.825708389282227
16.514677047729492
36.61735153

In [97]:
# See Output
model.eval()
mse = 0
with torch.no_grad():
    for values,labels in train_dat:
        values = values.reshape(-1,sequence_length,input_size).to(device)
        values = values.to(torch.float32)
        labels = labels.to(device)
        labels = labels.to(torch.float32)

        outputs = model(values)
        _, predicted = torch.max(outputs.data,1)
        print(predicted)
        print(labels)
        mse += np.sum((predicted-labels)**2)

print(mse)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])
tensor([ 0.3201,  0.8337,  3.4599, 10.3992,  0.6335,  0.2896,  7.5089,  4.4695,
         8.4194, 18.7016, 17.2056,  0.8775,  1.9219,  0.7886,  1.0361,  1.8945,
         3.1402, 11.2544, 10.5998, 23.9145,  5.6102, 15.1279, 19.9027,  2.8245,
         6.7379, 21.0713, 20.6244, 15.2556, 10.2007,  1.3615, 17.1013,  1.1815,
         0.3201, 13.9788,  1.3490,  1.5591,  0.3201,  3.7489,  1.8452,  2.2540,
        27.1625,  0.5988,  4.1117,  2.8245, 14.0348,  3.2650,  2.8245,  3.1066,
         0.6766, 26.9691, 15.5057,  2.1659,  1.9589,  0.3201,  3.2065,  1.0902,
         1.7929,  1.6506,  1.9237, 23.1998, 18.2980, 17.6155,  5.4831,  3.1020,
         3.6879, 11

TypeError: sum() received an invalid combination of arguments - got (axis=NoneType, out=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: axis, out
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)
